In [2]:
import time
import pandas as pd
import time
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

In [3]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)
driver.get('https:/1xbet.whoscored.com/')
driver.close()

In [11]:
def crawling_league_teams(region, tournaments, api_delay_term=5):

    """
    cawling league team_id and team name datas
    
    Arg :
        region : region id (from whoscored) for certain league ex) england --- 252, spain --- 206
        tournaments : tournament(or league) id ex) premier league --- 2,
    
    return :
        crawling league team_id, team_name datas belong team_id parameter
        return pandas dataframe columns=team_id, team_name
    

    """
    
    # connect webdriver
    url = "https://1xbet.whoscored.com/Regions/" + str(region) + "/Tournaments/" + str(tournaments)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)

    # wait get league team datas
    time.sleep(api_delay_term) 
    
    # make pandas dataframe
    team_df = pd.DataFrame(columns=["team_id","team_name"])

    # get team data
    teams = driver.find_element(By.XPATH, "//*[@id='standings-20934-content']")
    teams = teams.find_elements(By.CSS_SELECTOR, "a.team-link")
    
    for team in teams:
        team_name = team.get_attribute("innerHTML")
        team_id = team.get_attribute("href").split("/")[4]
        team_df.loc[len(team_df)] = {"team_id":team_id, "team_name":team_name }
        
    # close webdriver
    driver.close()
    
    return team_df

In [12]:
PL = crawling_league_teams(252, 2)

In [13]:
PL

,team_id,team_name
0,13,Arsenal
1,167,Manchester City
2,23,Newcastle
3,32,Manchester United
4,30,Tottenham
5,170,Fulham
6,26,Liverpool
7,211,Brighton
8,189,Brentford
9,15,Chelsea


## League Table 

Premier League table (2022-2023 season)
- URL = 'https://1xbet.whoscored.com/Regions/[region id]/Tournaments/[league id]'
- 252 (region id), 2 (league id)

In [18]:
URL = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2'

In [37]:
def league_table(URL, api_delay_term=5):
    """
    cwaling league table(seasonal statistics) 
    
    Args : 
        URL : league table URL
    
    Output :
        leauge table(dataframe)
    
 
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)

    time.sleep(api_delay_term)

    league_table_df = pd.DataFrame(columns=[
        "team_number", "team_name", "P", "W", "D", "L", "GF", "GA", "GD", "Pts"])
    elements = driver.find_element(By.CLASS_NAME, "standings").find_elements(By.CSS_SELECTOR, "tr")

    for element in elements:
        league_table_dict = { 
            "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("href").split("/")[4], 
            "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("textContent").split("\t")[0],     
            "P": element.find_elements(By.CSS_SELECTOR, "td")[1].get_attribute("textContent").split("\t")[0],
            "W": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0], 
            "D": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0],
            "L": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0], 
            "GF": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
            "GA": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0],
            "GD": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
            "Pts": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
        }
        league_table_df.loc[len(league_table_df)] = league_table_dict

    # close webdriver
    driver.close()
    return league_table_df

In [38]:
PL2223 = league_table(URL)

In [39]:
PL2223

,team_number,team_name,P,W,D,L,GF,GA,GD,Pts
0,13,Arsenal,17,14,2,1,40,14,+26,44
1,167,Manchester City,17,12,3,2,45,16,+29,39
2,23,Newcastle,18,9,8,1,32,11,+21,35
3,32,Manchester United,17,11,2,4,27,20,+7,35
4,30,Tottenham,18,10,3,5,37,25,+12,33
5,170,Fulham,19,9,4,6,32,28,+4,31
6,26,Liverpool,17,8,4,5,34,22,+12,28
7,211,Brighton,17,8,3,6,32,25,+7,27
8,189,Brentford,18,6,8,4,30,28,+2,26
9,15,Chelsea,18,7,4,7,21,21,0,25


In [40]:
def league_table_added(URL, api_delay_term=3):
    """
    crawling league table with additional features
    ex) shot per game, Tackles per game ... etc.
    
    Args : 
        URL : league table URL
        
    Output : 
        league table (data.frame)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(api_delay_term)
    
    league_table_df = pd.DataFrame(columns=[
        "team_name", "P", "W", "D", "L", "GF", "GA", "GD", "Pts"])
    elements = driver.find_elements(By.CLASS_NAME, 'standings')[0].find_elements(By.CSS_SELECTOR, "tr")
    
    for element in elements:
        league_table_dict = { 
            "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("textContent").split("\t")[0],     
            "P": element.find_elements(By.CSS_SELECTOR, "td")[1].get_attribute("textContent").split("\t")[0],
            "W": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0], 
            "D": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0],
            "L": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0], 
            "GF": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
            "GA": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0],
            "GD": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
            "Pts": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
        }
        league_table_df.loc[len(league_table_df)] = league_table_dict
    
    time.sleep(api_delay_term)
    
    starter = driver.find_elements(By.ID, "sub-navigation")
    starter = starter[0].find_elements(By.CSS_SELECTOR, "li")[2]
    starter.click()
    
    
    team_stat_df1 = pd.DataFrame(columns=[
        "team_name", "Goals", "Shots pg", "Yellow", "Red", "Poss%", "Pass%", 
        "A_Won", "Rating"
    ])
    elements = driver.find_elements(By.ID, "top-team-stats-summary-content")
    elements = elements[0].find_elements(By.CSS_SELECTOR, "tr")
    
    for element in elements:
        team_table_dict1 = { 
            "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("textContent").split("\t")[0].split('. ')[1],     
            "Goals": element.find_elements(By.CSS_SELECTOR, "td")[1].get_attribute("textContent").split("\t")[0],
            "Shots pg": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0], 
            "Yellow": element.find_elements(By.CSS_SELECTOR, "td")[3].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0], 
            "Red": element.find_elements(By.CSS_SELECTOR, "td")[3].find_elements(By.CSS_SELECTOR, "span")[1].get_attribute("textContent").split("\t")[0], 
            "Poss%": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0], 
            "Pass%": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0],
            "A_Won": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0],
            "Rating": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
        }
        team_stat_df1.loc[len(team_stat_df1)] = team_table_dict1
    
    element = driver.find_element(By.CSS_SELECTOR, "a[href='#stage-team-stats-defensive']")
    element.click()
    
    time.sleep(api_delay_term)
    
    
    team_stat_df2 = pd.DataFrame(columns=[
        "team_name", "Shoted pg", "Tackles pg", "Intercept pg", "Fouls pg", "Offsides pg"
    ])
    elements = driver.find_elements(By.ID, "statistics-team-table-defensive")
    elements = elements[0].find_elements(By.ID, "top-team-stats-summary-content")
    elements = elements[0].find_elements(By.CSS_SELECTOR, "tr")
    
    for element in elements:
        team_table_dict2 = { 
            "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("textContent").split("\t")[0].split('. ')[1],     
            'Shoted pg': element.find_elements(By.CSS_SELECTOR, "td")[1].get_attribute("textContent").split("\t")[0],
            'Tackles pg': element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
            'Intercept pg': element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
            'Fouls pg': element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0], 
            'Offsides pg': element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
        }
        team_stat_df2.loc[len(team_stat_df2)] = team_table_dict2
    
    element = driver.find_element(By.CSS_SELECTOR, "a[href='#stage-team-stats-offensive']")
    element.click()
    
    time.sleep(api_delay_term)
    
    team_stat_df3 = pd.DataFrame(columns=[
        "team_name", "Shots OT pg", "Dribbles pg", "Fouled pg"
    ])
    elements = driver.find_elements(By.ID, "statistics-team-table-offensive")
    elements = elements[0].find_elements(By.ID, "top-team-stats-summary-content")
    elements = elements[0].find_elements(By.CSS_SELECTOR, "tr")
    
    for element in elements: 
        team_table_dict3 = { 
            "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].get_attribute("textContent").split("\t")[0].split('. ')[1],     
            'Shots OT pg': element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
            'Dribbles pg': element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
            'Fouled pg': element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0], 
        }
        team_stat_df3.loc[len(team_stat_df3)] = team_table_dict3
        
        
    team_stat_df = pd.merge(league_table_df, team_stat_df1, how='left', on='team_name')
    team_stat_df = pd.merge(team_stat_df, team_stat_df2, how='left', on='team_name')
    team_stat_df = pd.merge(team_stat_df, team_stat_df3, how='left', on='team_name')
    
    # close webdriver
    driver.close()
    
    return team_stat_df

In [41]:
df = league_table_added(URL)

In [42]:
df.head()

,team_name,P,W,D,L,GF,GA,GD,Pts,Goals,...,A_Won,Rating,Shoted pg,Tackles pg,Intercept pg,Fouls pg,Offsides pg,Shots OT pg,Dribbles pg,Fouled pg
0,Arsenal,17,14,2,1,40,14,+26,44,40,...,11.2,6.86,8.2,14.9,7.3,10.2,1.4,5.7,8.2,11.4
1,Manchester City,17,12,3,2,45,16,+29,39,45,...,12.7,6.93,7.1,12.9,6.4,8.5,1.5,6.1,7.6,8.8
2,Newcastle,18,9,8,1,32,11,+21,35,32,...,14.3,6.82,11.1,16.7,9.1,11.1,2.3,5,5.9,10.3
3,Manchester United,17,11,2,4,27,20,+7,35,27,...,12.5,6.74,11.9,15.6,9.9,11.6,2.1,5.5,5.2,7.2
4,Tottenham,18,10,3,5,37,25,+12,33,37,...,13.4,6.75,14.3,15.9,9.3,10.1,1.8,5.8,6.6,8.2


In [43]:
#df.to_csv('PL2223_league_table.csv')